#### Leemos todos los archivos

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
movies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies") \
                  .filter(f"file_date = '{v_file_date}'")

In [0]:
countries_df = spark.read.format("delta").load(f"{silver_folder_path}/countries")

In [0]:
productions_countries_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_countries") \
                  .filter(f"file_date = '{v_file_date}'")

In [0]:
movies_companies_df = spark.read.format("delta").load(f"{silver_folder_path}/movies_companies") \
                  .filter(f"file_date = '{v_file_date}'")

In [0]:
productions_companies_df = spark.read.format("delta").load(f"{silver_folder_path}/productions_companies") \
                  .filter(f"file_date = '{v_file_date}'")

#### join "country" y "production_country"

In [0]:
country_prod_coun_df = countries_df.join(productions_countries_df,
                                         countries_df.country_id == productions_countries_df.country_id,
                                         "inner").select(countries_df.country_name, countries_df.country_id, productions_countries_df.movie_id)

#### join "productions_companies" y "movies_companies"

In [0]:
prod_comp_mov_comp_df = productions_companies_df.join(movies_companies_df,
                                                       productions_companies_df.company_id == movies_companies_df.company_id,
                                                       "inner").select(productions_companies_df.company_name, productions_companies_df.company_id, movies_companies_df.movie_id)

#### join "movies_df", "country_prod_coun_df" y "prod_comp_mov_comp_df"
- Filtrar las peliculas donde su fecha de lanzzamiento sea mayor o igual a 2010

In [0]:
movies_filter_df = movies_df.filter("year_release_date >= 2010")

In [0]:
result_movies_country_prod_company_df = movies_filter_df.join(country_prod_coun_df,
                                                               movies_filter_df.movie_id == country_prod_coun_df.movie_id,
                                                               "inner").join(prod_comp_mov_comp_df,
                                                               movies_filter_df.movie_id == prod_comp_mov_comp_df.movie_id,
                                                               "inner")

- agregar columna "created_date"

In [0]:
from pyspark.sql.functions import lit

results_df = result_movies_country_prod_company_df \
            .select(movies_filter_df.movie_id, "country_id","title", "company_id", "budget", "revenue", "duration_time", "release_date", "country_name", "company_name") \
            .withColumn("created_date", lit(v_file_date))

- ordenar por la columna "title" de manera ascendente

In [0]:
results_order_by_df = results_df.orderBy(results_df.title.asc())

#### Escribir datos en el dataLake en formato Delta

In [0]:
#overwrite_partition(results_order_by_df, "movie_gold", "results_country_prod_company", "created_date")

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.company_id = src.company_id AND tgt.created_date = src.created_date'
merge_delte_lake(results_order_by_df, "movie_gold", "results_country_prod_company", gold_folder_path, merge_condition, "created_date")


In [0]:
%sql
SELECT * FROM movie_gold.results_country_prod_company

In [0]:
display(spark.read.format("delta").load(f"{gold_folder_path}/results_country_prod_company"))